<a href="https://colab.research.google.com/github/EmilisGit/Deep_learning/blob/main/t5_eff_small_fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.47.0 datasets==3.2.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
#@title Import Libraries
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import transformers
import os
import datasets
import tensorflow as tf
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

print("Transformers " + transformers.__version__)
print("Datasets " + datasets.__version__)

Transformers 4.47.0
Datasets 3.2.0


# Import Dataset
[ambrosfitz/10k_history_summary](https://huggingface.co/datasets/ambrosfitz/10k_history_summary?row=99)

In [3]:
hfDataset = load_dataset("ambrosfitz/10k_history_summary")
split = hfDataset['train'].train_test_split(test_size=0.2)
hfDataset['train'] = split['train'].shuffle(seed=42).select(range(3000))
hfDataset['test'] = split['test'].shuffle(seed=42).select(range(300))

hfDataset.save_to_disk("/content/hfDataset")
hfDataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/288 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10177 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'target'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'target'],
        num_rows: 300
    })
})

In [4]:
INPUT_NAME = "text"
OUTPUT_NAME = "target"
hfDataset['train'][0]

{'text': "In the aftermath of the 1973 Arab-Israeli War, Secretary of State Henry Kissinger skillfully navigated diplomatic waters to ease pressure on Israel for a complete withdrawal from occupied Arab territories. By orchestrating a deal that required only minor Israeli concessions on the Sinai Peninsula and Golan Heights, Kissinger effectively removed Egypt from the conflict. This move allowed Israel to solidify its hold on Syrian and Palestinian lands, making a return to the 1967 borders unlikely. While this diplomatic success bolstered Israel's position and weakened the Soviet Union in the Middle East, it also strained America's relations with many Arab nations, leading to new threats to the United States. The delicate balance Kissinger achieved in the region ultimately came at the expense of enduring peace.",
 'target': "Post-1973 Arab-Israeli War, Kissinger's diplomacy favored Israel, easing pressure for full withdrawal from Arab lands. He brokered a deal with Egypt, removing it

In [5]:
BASE_MODEL = "google/t5-efficient-small"
model = TFAutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/242M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at google/t5-efficient-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

#Duomenų analizė
Įvertinkime, ar tekstų ilgiai priimtini mūsų modeliui ir apmokomi naudojant colab aplinką.

In [6]:
import numpy as np
import math

input_lengths = [len(tokenizer.encode(sentence)) for sentence in hfDataset['train'][INPUT_NAME]]
output_lengths = [len(tokenizer.encode(sentence)) for sentence in hfDataset['train'][OUTPUT_NAME]]
print(f"Average input length: {math.floor(np.mean(input_lengths))}")
print(f"Max input length: {np.max(input_lengths)}")
print(f"Average output length: {math.floor(np.mean(output_lengths))}")
print(f"Max output length: {np.max(output_lengths)}")


Average input length: 219
Max input length: 540
Average output length: 83
Max output length: 245


In [7]:
TEST_SAMPLE = hfDataset['test'][INPUT_NAME][1]
TEST_SAMPLE

"In the aftermath of the race riots in Detroit in 1943, the response from the city's white leaders was characterized by ambivalence and reluctance to intervene effectively. Despite their disapproval of the violence, they hesitated to call for the National Guard until the rioting posed a threat to the entire community, not just the black residents. The riots of that time reflected interracial conflicts driven by whites, including European immigrants and uprooted southerners, as aggressors against black victims who were themselves newcomers. The violence stemmed from a lack of understanding on how to achieve their goals through legitimate means, leading to reactionary attempts to maintain the racial status quo amidst rapid social change. Although these riots may have had short-term impacts, they ultimately proved unsuccessful due to the powerful economic, social, and political shifts underlying the migration of southern blacks and the growing militancy of northern blacks."

#Bazinis modelis
Apibrėžiame teksto apibendrinimo funkciją Summarize_text()
Panaudojame šią funkciją ant bazinio modelio, be fine-tuningo, kad įvertintume pradinį modelio rezultatą.

In [8]:
def Summarize_text(text):
    inference_text = tokenizer("Summarize: " + text, return_tensors="tf", max_length=512, padding="max_length", truncation=True)

    outputs = model.generate(
        input_ids=inference_text["input_ids"],
        attention_mask=inference_text["attention_mask"],
        max_length=128,
        num_beams=3,
        early_stopping=True
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text
Summarize_text(TEST_SAMPLE)

'. Summarize: The race riots in Detroit in 1943 reflected a lack of understanding of race riots in Detroit in 1943, the race riots in Detroit in 1943. the race riots in Detroit in 1943. the race riots in Detroit in 1943 the race riots.  the race::::.'

#Duomenų apdorojimas
Apibrėžiame duomenų apdorojimo funkciją preprocess_data()
Funkcija reikalinga užkoduoti tekstą (tokenizuoti), kad jis būtų galimas panaudoti modelio fine-tuningui. Čia taip pat nustatome modelio konteksto lauko dydį, šiuo atvėju 512 tokenų ir generuojamų tokenų ilgį - 128. Šie ilgiai atitinka summarizavimo užduoties reikalavimus, tinka mūsų duomenims ir telpa į modelio apribojimus.

In [9]:
def preprocess_data(examples):
    inputs = ["Summarize: " + ex for ex in examples[INPUT_NAME]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["input_ids"] = [inp for inp in model_inputs["input_ids"]]

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples[OUTPUT_NAME], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

encoded_dataset = hfDataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Kiek turime apmokymo duomenų (mėginių)?

In [10]:
print("Apmokymo duomenu: ", len(hfDataset['train']))
print("Testavimo duomenu: ", len(hfDataset['test']))

Apmokymo duomenu:  3000
Testavimo duomenu:  300


Su .ipynb dokumentu, jau suteiktas apmokytas modelis Summarization_t5_eff_small.keras. **Pakeisti FINE_TUNED_MODEL_PATH**, pagal jūsų modelio vietą diske.



In [11]:
from google.colab import drive
drive.mount('/content/drive')

FINE_TUNED_MODEL_PATH = "/content/drive/MyDrive/Colab/Summarization_t5_eff_small.keras"

if(not os.path.exists(FINE_TUNED_MODEL_PATH)):
  training_data = encoded_dataset["train"]
  data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

  tf_train_dataset = training_data.to_tf_dataset(
      columns=["input_ids", "attention_mask", "labels"],
      collate_fn=data_collator,
      shuffle=True,
      batch_size=8,
  )

  model.compile(optimizer="adam")
  model.fit(tf_train_dataset, epochs=3)
  model.save_pretrained("/content/drive/MyDrive/Colab/Summarization_t5_eff_small_v2.keras")
else:
  model = TFAutoModelForSeq2SeqLM.from_pretrained(FINE_TUNED_MODEL_PATH)

Mounted at /content/drive


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/MyDrive/Colab/Summarization_t5_eff_small.keras.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Pastebekime kad po tokenizavimo mūsų `Dataset` objectas gavo naujas savybes (stulpelius, raktažodžius) - `'input_ids'`, `'token_type_ids'` ir `'attention_mask'`:

In [13]:
TEST_SAMPLE

"In the aftermath of the race riots in Detroit in 1943, the response from the city's white leaders was characterized by ambivalence and reluctance to intervene effectively. Despite their disapproval of the violence, they hesitated to call for the National Guard until the rioting posed a threat to the entire community, not just the black residents. The riots of that time reflected interracial conflicts driven by whites, including European immigrants and uprooted southerners, as aggressors against black victims who were themselves newcomers. The violence stemmed from a lack of understanding on how to achieve their goals through legitimate means, leading to reactionary attempts to maintain the racial status quo amidst rapid social change. Although these riots may have had short-term impacts, they ultimately proved unsuccessful due to the powerful economic, social, and political shifts underlying the migration of southern blacks and the growing militancy of northern blacks."

In [12]:
Summarize_text(TEST_SAMPLE)

'Post-WWII riots in Detroit, driven by ambivalence and reluctance to intervene effectively. Despite their disapproval, they hesitated to call for the National Guard until the riot posed a threat to the entire community, not just black residents. The violence stemmed from interracial conflicts driven by European immigrants and southerners, driven by interracial conflicts driven by whites, including European immigrants and uprooted southerners'